# Build a Customer Support Question Answering Chatbot

## Introduction

As we witness accelerated technological progress, large language models like GPT-4 and ChatGPT have emerged as significant breakthroughs in the tech landscape. These state-of-the-art models demonstrate exceptional prowess in content generation. However, they are not without their share of challenges, such as biases and hallucinations. Despite these limitations, LLMs have the potential to bring about a transformative impact on chatbot development.

Traditional, primarily intent-based chatbots have been designed to respond to specific user intents. These intents comprise a collection of sample questions and corresponding responses. For instance, a "Restaurant Recommendations" intent might include sample questions like "Can you suggest a good Italian restaurant nearby?" or "Where can I find the best sushi in town?" with responses such as "You can try the Italian restaurant 'La Trattoria' nearby" or "The top-rated sushi place in town is 'Sushi Palace.'"

## Workflow

This project aims to build a chatbot that leverages GPT3 to search for answers within documents. The workflow for the experiment is explained in the following diagram.

First we scrape some content from online articles, we split them into small chunks, compute their embeddings and store them in Deep Lake. Then, we use a user query to retrieve the most relevant chunks from Deep Lake, we put them into a prompt, which will be used to generate the final answer by the LLM.

It is important to note that there is always a risk of generating hallucinations or false information when using LLMs. Although this might not be acceptable for many customers support use cases, the chatbot can still be helpful for assisting operators in drafting answers that they can double-check before sending them to the user.

In the next steps, we'll explore how to manage conversations with GPT-3 and provide examples to demonstrate the effectiveness of this workflow:

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain.document_loaders import SeleniumURLLoader, WebBaseLoader

from langchain import PromptTemplate

In [11]:
# we'll use information from the following articles
urls = ['https://beebom.com/what-is-nft-explained/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-download-gif-twitter/',
        'https://beebom.com/how-use-chatgpt-linux-terminal/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-save-instagram-story-with-music/',
        'https://beebom.com/how-install-pip-windows/',
        'https://beebom.com/how-check-disk-usage-linux/']

## 1: Split the documents into chunks and compute their embeddings

We load the documents from the provided URLs and split them into chunks using the CharacterTextSplitter with a chunk size of 1000 and no overlap:

In [14]:
# use the selenium scraper to load the documents
#loader = SeleniumURLLoader(urls=urls)
loader = WebBaseLoader(urls)
docs_not_splitted = loader.load()

print('Documents:', len(docs_not_splitted))

# we remove the documents with no text, if using Selenium
#docs = [doc for doc in docs_not_splitted if doc.text]

# we split the documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs_not_splitted)

print('Documents split:', len(docs))

Created a chunk of size 1212, which is longer than the specified 1000


Documents: 8
Documents split: 141


Next, we compute the embeddings using OpenAIEmbeddings and store them in a Deep Lake vector store on the cloud. In an ideal production scenario, we could upload a whole website or course lesson on a Deep Lake dataset, allowing for search among even thousands or millions of documents. As we are using a cloud serverless Deep Lake dataset, applications running on different locations can easily access the same centralized dataset without the need of deploying a vector store on a custom machine.

In [15]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
my_activeloop_org_id = "edumunozsala"
my_activeloop_dataset_name = "langchain_course_customer_support"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

c:\Users\edumu\anaconda3\envs\llm-lc\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.0) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


Your Deep Lake dataset has been successfully created!
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/edumunozsala/langchain_course_customer_support
hub://edumunozsala/langchain_course_customer_support loaded successfully.


Evaluating ingest: 100%|██████████| 1/1 [00:15<00:00


Dataset(path='hub://edumunozsala/langchain_course_customer_support', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (141, 1536)  float32   None   
    ids      text     (141, 1)      str     None   
 metadata    json     (141, 1)      str     None   
   text      text     (141, 1)      str     None   


['d55d3674-6b69-11ee-ab11-cc2f714963ed',
 'd55d3675-6b69-11ee-bbcd-cc2f714963ed',
 'd55d3676-6b69-11ee-9512-cc2f714963ed',
 'd55d3677-6b69-11ee-a5a5-cc2f714963ed',
 'd55d3678-6b69-11ee-b59f-cc2f714963ed',
 'd55d3679-6b69-11ee-a5d1-cc2f714963ed',
 'd55d367a-6b69-11ee-a698-cc2f714963ed',
 'd55d367b-6b69-11ee-a576-cc2f714963ed',
 'd55d367c-6b69-11ee-a3f3-cc2f714963ed',
 'd55d367d-6b69-11ee-a881-cc2f714963ed',
 'd55d367e-6b69-11ee-8abb-cc2f714963ed',
 'd55d367f-6b69-11ee-84ec-cc2f714963ed',
 'd55d3680-6b69-11ee-83fa-cc2f714963ed',
 'd55d3681-6b69-11ee-8f73-cc2f714963ed',
 'd55d3682-6b69-11ee-a7ee-cc2f714963ed',
 'd55d3683-6b69-11ee-913c-cc2f714963ed',
 'd55d3684-6b69-11ee-9d43-cc2f714963ed',
 'd55d3685-6b69-11ee-a873-cc2f714963ed',
 'd55d3686-6b69-11ee-a0a4-cc2f714963ed',
 'd55d3687-6b69-11ee-992f-cc2f714963ed',
 'd55d3688-6b69-11ee-ab50-cc2f714963ed',
 'd55d3689-6b69-11ee-967b-cc2f714963ed',
 'd55d368a-6b69-11ee-80c7-cc2f714963ed',
 'd55d368b-6b69-11ee-ad95-cc2f714963ed',
 'd55d368c-6b69-

To retrieve the most similar chunks to a given query, we can use the similarity_search method of the Deep Lake vector store:

In [16]:
# let's see the top relevant documents to a specific query
query = "how to check disk usage in linux?"
docs = db.similarity_search(query)

# print the content of the first document
print(docs[0].page_content)

Check Disk Usage Using Gnome Disk ToolCheck Disk Usage Using Disk Usage Analyzer ToolCleanup Disk using Disk Usage Analyzer


Check Disk Space Using the df Command
In Linux, there are many commands to check disk usage, the most common being the df command. The df stands for “Disk Filesystem” in the command, which is a handy way to check the current disk usage and the available disk space in Linux. The syntax for the df command in Linux is as follows:



df <options> <file_system>
The options to use with the df command are:


## 2: Craft a prompt for GPT-3 using the suggested strategies

We will create a prompt template that incorporates role-prompting, relevant Knowledge Base information, and the user's question:

In [18]:
# let's write a prompt for a customer support chatbot that
# answer questions using information extracted from our db
template = """You are an exceptional customer support chatbot that gently answer questions.

You know the following context information.

{chunks_formatted}

Answer to the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {query}

Answer:"""

prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template,
)

## 3: Utilize the GPT3 model with a temperature of 0 for text generation

To generate a response, we first retrieve the top-k (e.g., top-3) chunks most similar to the user query, format the prompt, and send the formatted prompt to the GPT3 model with a temperature of 0.

In [19]:
# the full pipeline

# user question
query = "How to check disk usage in linux?"

# retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)

# generate answer
llm = OpenAI(model="text-davinci-003", temperature=0)
answer = llm(prompt_formatted)
print(answer)

 You can check disk usage in Linux using the df command. This command stands for “Disk Filesystem” and is a handy way to check the current disk usage and the available disk space in Linux. The syntax for the df command in Linux is as follows: df <options> <file_system>. Additionally, you can also use the du command to check disk usage in Linux. This command stands for “Disk Usage” and is used to display disk occupancy of a particular type or disk usage for a particular directory. You can also use a GUI application such as Gnome Disk Tool or Disk Usage Analyzer to check disk usage in Linux.
